In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
table_name = ''
watermark_value = ''
Src_type = ''

StatementMeta(, 83c3cd6d-999d-498c-b862-7a21fc72bc07, 4, Finished, Available, Finished)

In [3]:
import com.microsoft.spark.fabric
from datetime import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *

StatementMeta(, 83c3cd6d-999d-498c-b862-7a21fc72bc07, 5, Finished, Available, Finished)

In [4]:
watermark_date = datetime.strptime(watermark_value, "%Y-%m-%d").date()
if Src_type == 'Snow':
    filtered_df = spark.table(f"dbo.{table_name}") \
        .withColumn("SRC_TRANSACTION_DATE", to_date(col("SRC_TRANSACTION_DATE"), "dd-MM-yyyy")) \
        .filter(col("SRC_TRANSACTION_DATE") > watermark_date)

    filtered_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"BRONZE.{table_name}")

    max_date = str(filtered_df.agg(max("SRC_TRANSACTION_DATE")).collect()[0][0])

    config_df = spark.read.synapsesql("Capstone.dbo.Config_table") \
                .withColumn(
                    "WATERMARK_COLUMN_VALUE",
                    when((col("TGT_TABLE_NAME") == table_name) & (lit(max_date)!='None'), max_date).otherwise(col("WATERMARK_COLUMN_VALUE"))
                )

    config_df.write.mode('overwrite').synapsesql("Capstone.dbo.Config_table")

    mssparkutils.notebook.exit({"updated_watermark_date": config_df.filter(col("TGT_TABLE_NAME") == table_name).select('WATERMARK_COLUMN_VALUE').collect()[0][0]})

else:
    spark.table(f"dbo.{table_name}").write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"Bronze.{table_name}")
 
 

StatementMeta(, 83c3cd6d-999d-498c-b862-7a21fc72bc07, 6, Finished, Available, Finished)

In [2]:
#Full load

source_schema = "dbo"
new_schema = "BRONZE"
 
# Create new schema
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {new_schema}")
 
# List all tables
tables_df = spark.sql(f"SHOW TABLES IN {source_schema}")
 
# Loop through and copy each table
for row in tables_df.collect():
    table_name = row['tableName']
    full_source_table = f"{source_schema}.{table_name}"
    full_target_table = f"{new_schema}.{table_name}"
 
    print(f"Copying table: {full_source_table} --> {full_target_table}")
 
    df = spark.table(full_source_table)
    df.write.mode("overwrite").saveAsTable(full_target_table)
 
# Verify
spark.sql(f"SHOW TABLES IN {new_schema}").show()

StatementMeta(, 1590ab27-a3e9-4c21-b227-c7c9c271ecac, 4, Finished, Available, Finished)

Copying table: dbo.company_affiliation --> BRONZE.company_affiliation
Copying table: dbo.connection_growth --> BRONZE.connection_growth
Copying table: dbo.job_applications --> BRONZE.job_applications
Copying table: dbo.linkedin_users --> BRONZE.linkedin_users
Copying table: dbo.post --> BRONZE.post
Copying table: dbo.post_performance --> BRONZE.post_performance
Copying table: dbo.user_activity --> BRONZE.user_activity
Copying table: dbo.user_identity --> BRONZE.user_identity
+--------------------+-------------------+-----------+
|           namespace|          tableName|isTemporary|
+--------------------+-------------------+-----------+
|LinkedIN.LinkedIN...|company_affiliation|      false|
|LinkedIN.LinkedIN...|  connection_growth|      false|
|LinkedIN.LinkedIN...|   job_applications|      false|
|LinkedIN.LinkedIN...|     linkedin_users|      false|
|LinkedIN.LinkedIN...|               post|      false|
|LinkedIN.LinkedIN...|   post_performance|      false|
|LinkedIN.LinkedIN...|   